```
Copyright 2023 테디노트(https://teddylee777.github.io)
teddylee777@gmail.com
무단 전재 및 재배포 금지

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
    
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```


## 환경설정

In [ ]:
# 필요한 라이브러리 설치
# pip install langchain openai 

In [ ]:
# OPENAI_API
# import os

# os.environ['OPENAI_API_KEY'] = 'OPENAI API KEY 입력'

In [1]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

True

## 웹문서 요약

### WebBaseLoader

웹사이트에서 본문의 내용을 크롤링해주는 모듈

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate


# 네이버 뉴스기사 주소
url = 'https://n.news.naver.com/mnews/article/025/0003311700?sid=101'
# 웹 문서 크롤링
loader = WebBaseLoader(url)

# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(        
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

# 웹사이트 내용 크롤링 후 Chunk 단위로 분할
docs = WebBaseLoader(url).load_and_split(text_splitter)
docs

[Document(page_content='수출 살아나나…반도체 바닥 찍고, 무역흑자 2년래 최고\n\n본문 바로가기\n\n\nNAVER\n\n뉴스\n\n\n연예\n\n\n스포츠\n\n\n날씨\n\n\n프리미엄\n\n\n검색\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n구독\n\n중앙일보 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n중앙일보 언론사 구독 해지되었습니다.\n\n\n수출 살아나나…반도체 바닥 찍고, 무역흑자 2년래 최고\n\n\n입력2023.10.03. 오전 12:06\n\n\n수정2023.10.03. 오전 12:07\n\n기사원문\n \n\n\n정종훈 기자\n\n\n정종훈 기자\n\n구독\n구독중\n\n\n구독자\n0\n\n\n응원수\n0\n\n더보기\n\n\n추천\n\n\n쏠쏠정보\n0\n\n\n흥미진진\n0\n\n\n공감백배\n0\n\n\n분석탁월\n0\n\n\n후속강추\n0\n\n\n \n\n댓글\n\n본문 요약봇\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n텍스트 음성 변환 서비스 사용하기\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\nSNS 보내기\n\n인쇄하기\n\n\t\t\t  반도체·중국의 수출 감소율은 연중 최소 폭, 무역수

### 템플릿 정의

뉴스기사의 본문의 내용은 LLM 모델이 허용하는 **최대 토큰의 수를 넘길 가능성이 크므로** , 쪼개서 여러번 질의 하는 방식인 `map_reduce` 방식으로 질의하도록 옵션 설정합니다. `stuff` 와 `map_reduce` 의 차이는 아래에서 다룹니다.

- `prompt`: 쪼개진 Chunk 단위의 프롬프트
- `combine_prompt`: 전체 문서에 대한 프롬프트

아래의 프롬프트 템플릿에서 `{text}` 변수에는 웹에서 긁어온 정보가 대입됩니다.

In [12]:
# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 신문기사의 제목
주요내용: 한 줄로 요약된 내용
작성자: 김철수 대리
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])

### LLM 객체 생성

질의에 사용할 LLM 객체를 생성합니다.

In [13]:
# LLM 객체 생성
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo-16k-0613')

### load_summarize_chain

`chain_type` 매개변수

- `stuff` 타입: 모든 프롬프트 텍스트를 단일 호출로 LLM에 보냅니다. 아쉽게도 이 옵션은 큰 문서에 대해 작동하지 않습니다.
- `map_reduce` 타입: 프롬프트 텍스트를 여러 청크로 분할하여 LLM에 전송하고, 마지막에 청크 결과를 결합합니다. 이는 큰 문서를 요약하는 한 가지 방법이지만, LLM에 여러 번의 호출이 필요합니다. 이는 정확도와 성능에 영향을 미칠 수 있습니다.

In [14]:
# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm, 
                             map_prompt=prompt, 
                             combine_prompt=combine_prompt, 
                             chain_type='map_reduce', 
                             verbose=False)

# 실행결과
print(chain.run(docs))

제목: 수출 회복세 지속, 중국 수출 감소율 최소 폭 기록

주요내용: 한 달째 한 자릿수 감소율로 역성장한 수출과 무역흑자가 2년래 최고를 기록하며 수출이 살아나고 있다. 반도체 수출은 바닥을 찍고, 중국으로의 수출 감소율도 최소 폭을 기록했다. 미국과 유럽연합으로의 수출도 증가세를 보이며 수출 회복이 예상된다. 그러나 중국의 반등과 국제유가 상승, 글로벌 고금리 등의 위험 요소가 존재한다.

작성자: 김철수 대리

내용:
- 한 달째 한 자릿수 감소율로 역성장한 수출과 무역흑자가 2년래 최고를 기록하며 수출이 살아나고 있다.
- 반도체 수출은 바닥을 찍고, 중국으로의 수출 감소율도 최소 폭을 기록했다.
- 미국과 유럽연합으로의 수출도 증가세를 보이며 수출 회복이 예상된다.
- 그러나 중국의 반등과 국제유가 상승, 글로벌 고금리 등의 위험 요소가 존재한다.


## 전체코드

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate

# 네이버 뉴스기사 주소
url = 'https://n.news.naver.com/article/437/0000361628?cds=news_media_pc'

# 웹 문서 크롤링
loader = WebBaseLoader(url)

# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(        
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

# 웹사이트 내용 크롤링 후 Chunk 단위로 분할
docs = WebBaseLoader(url).load_and_split(text_splitter)

# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 신문기사의 제목
주요내용: 한 줄로 요약된 내용
작성자: 김철수 대리
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])

# LLM 객체 생성
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo-16k')

# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm, 
                             map_prompt=prompt, 
                             combine_prompt=combine_prompt, 
                             chain_type="map_reduce", 
                             verbose=False)


In [19]:
# 실행결과
print(chain.run(docs))

제목: 한국 야구 대표팀 대만전에서 완패, 슈퍼라운드 진출 위해 일본과 중국과 경기해야 함
주요내용: 한국 야구 대표팀이 대만에 0-4로 완패하여 아시안게임 4연패 목표에 빨간불이 켜졌다. 대만은 2승을 거두어 B조 1위를 예약하였고, 한국은 1승 1패를 기록하여 슈퍼라운드 진출을 위해 일본과 중국을 이겨야 한다.
작성자: 김철수 대리
내용:
- 한국 야구 대표팀이 대만에 0-4로 완패하여 아시안게임 4연패 목표에 빨간불이 켜졌다.
- 대만은 2승을 거두어 B조 1위를 예약하였고, 한국은 1승 1패를 기록하여 슈퍼라운드 진출을 위해 일본과 중국을 이겨야 한다.
